# importando bilbiotecas e a base de dados CSV
- bibliotecas utilizadas: Pandas, streamlit e plotly


In [51]:
import pandas as pd
import plotly.express as px

base = "Ecommerce_DBS.csv"

df = pd.read_csv(base)

df.head()

,Customer ID,Purchase Date,Product Category,Product Price,Quantity,Total Purchase Amount,NPS,Customer Age,Gender,Source,Country,State,Latitude,Longituide
0,46251,08/09/2020,Electronics,12,3,740,7,20,Male,Instagram Campign,Canada,Alberta,55.000000,-115.000000
1,46251,05/03/2022,Home,468,4,2739,8,20,Male,Instagram Campign,Canada,Ontario,50.000000,-85.000000
2,46251,23/05/2022,Home,288,2,3196,10,20,Male,SEM,United States,New Mexico,34.840515,-106.248482
3,46251,12/11/2020,Clothing,196,1,3509,3,20,Male,Instagram Campign,Canada,Saskatchewan,55.000000,-106.000000
4,13593,27/11/2020,Home,449,1,3452,3,20,Female,Instagram Campign,United States,California,36.116203,-119.681564


# visualizando alguns dados

In [52]:
# preço total de vendas por categoria
vendas_por_produto = df.groupby("Product Category")["Total Purchase Amount"].sum().sort_values(ascending=False)
print("\nValor total de vendas por Categoria:")
print(vendas_por_produto)

# soma total de NPS por categoria
nps_por_produto = df.groupby("Product Category")["NPS"].sum().sort_values(ascending=False)
print("\nValor total de NPS por Categoria:")
print(nps_por_produto)

# Quantidade de unidades vendidas por categoria
qtd_por_produto = df.groupby("Product Category")["Quantity"].sum().sort_values(ascending=False)
print("\nQuantidade de unidades vendidas por Categoria:")
print(qtd_por_produto)

# soma total do preço por categoria
soma_por_produto = df.groupby("Product Category")["Product Price"].sum().sort_values(ascending=False)
print("\nSoma Total do Preço por Categoria:")
print(soma_por_produto)


Valor total de vendas por Categoria:
Product Category
Books          204939601
Clothing       204532405
Electronics    136599467
Home           135271210
Name: Total Purchase Amount, dtype: int64

Valor total de NPS por Categoria:
Product Category
Books          374837
Clothing       374052
Home           249823
Electronics    249246
Name: NPS, dtype: int64

Quantidade de unidades vendidas por Categoria:
Product Category
Clothing       225322
Books          223876
Electronics    150828
Home           149698
Name: Quantity, dtype: int64

Soma Total do Preço por Categoria:
Product Category
Clothing       19096701
Books          19080772
Electronics    12783292
Home           12704113
Name: Product Price, dtype: int64


# tratando tabela
- passo 1: tirando colunas que não serão utilizadas
- passo 2: arrumando os valores da coluna 'Total Purchase Amount' (multiplicando o valor das colunas 'Product Price' e 'Quantity')
- passo 3: passando a coluna 'Purchase Date' de String para DateTime
- passo 4: adicionando coluna 'Purchase Year'
- passo 5: filtrando os ultimos 3 anos
- passo 6: agrupando por ano
- passo 7: responder à pergunta "Qual o produto mais caro e o mais barato?"
- passo 8: responder à pergunta "Quais produtos mais vendidos considerando os últimos 3 anos?"
- passo 9: responder à pergunta "Qual o produto com o melhor e pior NPS?"
- passo 10: responder à pergunta "Qual categoria de produto mais e menos vendida?"
- passo 11: responder à pergunta "Qual categoria mais e menos cara?"
- passo 12: responder à pergunta "Qual o canal ideal para vender determinado tipo de produto (considerando genero e idade)?"

In [53]:
# passo 1: tirando colunas que não serão utilizadas
df = df.drop(["Customer ID", "Country", "State", "Latitude", "Longituide"], axis=1)

In [54]:
# passo 2: arrumando a coluna 'Total Purchase Amount' (multiplicando o valor das colunas 'Product Price' e 'Quantity')
df["Total Purchase Amount"] = df["Product Price"] * df["Quantity"]

In [55]:
# passo 3: passando a coluna 'Purchase Date' de String para DateTime
df["Purchase Date"] = pd.to_datetime(df["Purchase Date"], format='mixed')
print(df.dtypes)

Purchase Date            datetime64[ns]
Product Category                 object
Product Price                     int64
Quantity                          int64
Total Purchase Amount             int64
NPS                               int64
Customer Age                      int64
Gender                           object
Source                           object
dtype: object


In [56]:
# passo 4: adicionando coluna 'Purchase Year'
df["Purchase Year"] = df["Purchase Date"].dt.year
df["Purchase Date"].describe()

count                           250000
mean     2021-11-10 06:23:53.894400256
min                2020-01-01 00:00:00
25%                2020-12-02 00:00:00
50%                2021-11-07 00:00:00
75%                2022-10-11 00:00:00
max                2023-12-09 00:00:00
Name: Purchase Date, dtype: object

In [57]:
# passo 5: filtrando os ultimos 3 anos
ultimos_3_anos = df[df["Purchase Year"]>2020]
ultimos_3_anos.describe()

,Purchase Date,Product Price,Quantity,Total Purchase Amount,NPS,Customer Age,Purchase Year
count,181955,181955.000000,181955.000000,181955.000000,181955.00000,181955.000000,181955.000000
mean,2022-05-14 20:18:11.584183040,254.774488,2.998131,763.642725,4.99191,43.739216,2021.893314
min,2021-01-01 00:00:00,10.000000,1.000000,10.000000,0.00000,18.000000,2021.000000
25%,2021-09-06 00:00:00,132.000000,2.000000,288.000000,2.00000,30.000000,2021.000000
50%,2022-05-10 00:00:00,255.000000,3.000000,604.000000,5.00000,44.000000,2022.000000
75%,2023-01-15 00:00:00,378.000000,4.000000,1132.000000,8.00000,57.000000,2023.000000
max,2023-12-09 00:00:00,500.000000,5.000000,2500.000000,10.00000,70.000000,2023.000000
std,NaN,141.626335,1.415840,591.755809,3.16844,15.380347,0.787064


In [58]:
# passo 6: agrupando por ano
agrupamento_ano = df.sort_values(by="Purchase Year")
df = agrupamento_ano
agrupamento_ano.head()

,Purchase Date,Product Category,Product Price,Quantity,Total Purchase Amount,NPS,Customer Age,Gender,Source,Purchase Year
0,2020-08-09,Electronics,12,3,36,7,20,Male,Instagram Campign,2020
119437,2020-11-10,Electronics,73,4,292,9,70,Female,Organic Search,2020
119429,2020-06-24,Home,295,1,295,8,46,Female,Organic Search,2020
119426,2020-05-10,Home,20,5,100,6,55,Female,Instagram Campign,2020
119423,2020-06-21,Electronics,88,3,264,3,63,Female,Instagram Campign,2020


In [59]:
# passo 7: responder à pergunta "Qual o produto mais caro e o mais barato?"

# indice para o produto mais caro e mais barato
id_max = df["Product Price"].idxmax()
id_min = df["Product Price"].idxmin()

prod_mais_caro = df.loc[id_max]
prod_mais_barato = df.loc[id_min]

# resultado
print("produto mais caro:", prod_mais_caro)
print("produto mais barato:", prod_mais_barato)

produto mais caro: Purchase Date            2020-03-11 00:00:00
Product Category                       Books
Product Price                            500
Quantity                                   1
Total Purchase Amount                    500
NPS                                        7
Customer Age                              31
Gender                                Female
Source                     FaceBook campaign
Purchase Year                           2020
Name: 120655, dtype: object
produto mais barato: Purchase Date            2020-02-03 00:00:00
Product Category                 Electronics
Product Price                             10
Quantity                                   4
Total Purchase Amount                     40
NPS                                        2
Customer Age                              56
Gender                                  Male
Source                        Organic Search
Purchase Year                           2020
Name: 119683, dtype: object


In [60]:
# passo 8: responder à pergunta "Quais produtos mais vendidos considerando os últimos 3 anos?"

# obter o ano atual
ano_atual = 2024

# filtrar os dados para os ultimos 3 anos
df_filtro = df[df["Purchase Year"] >= (ano_atual - 3)]
df_filtro

# agrupar os dados por categoria e calcular a quantidade total vendida nos ultumos 3 anos
df_agrupado = df_filtro.groupby("Product Category")["Quantity"].sum().reset_index()

# ordenar os produtos pela quantidade total vendida em ordem decrescente
df_agrupado = df_agrupado.sort_values(by="Quantity", ascending=False)
df_agrupado

# grafico de barras
fig = px.bar(df_agrupado, x="Product Category", y="Quantity", title="Produtos mais vendidos nos ultimos 3 anos",
             labels={"Product Category": "Categoria", "Quantity": "Quantidade Vendida"},
             color="Quantity", color_continuous_scale=px.colors.sequential.Viridis)

fig.update_layout(xaxis_title="Product Category", yaxis_title="Quantity Sell")
fig.show()

In [61]:
# passo 9: responder à pergunta "Qual o produto com o melhor e pior NPS?"

# calculo de média de NPS para cada categoria e mostrar apenas 2 casas decimais
df_nps = df.groupby("Product Category")["NPS"].mean().round(2).reset_index()
df_nps = df_nps.sort_values(by="NPS", ascending=False)
df_nps

# encontrar o produto com a melhor média de NPS
melhor_nps = df_nps.iloc[0]

# encontrar o produto com a pior média de NPS
pior_nps = df_nps.iloc[-1]

# exibir resultados
print("\nProduto com a melhor média de NPS:")
print(melhor_nps)

print("\nProduto com a pior média de NPS:")
print(pior_nps)

# gráfico de barras para média de NPS
fig = px.bar(df_nps, x="Product Category", y="NPS", title="Média de NPS dos Produtos",
             labels={"Product Category": "Categoria", "NPS": "Média de NPS"},
             color="NPS", color_continuous_scale=px.colors.sequential.Viridis)

fig.update_layout(xaxis_title="Product Category", yaxis_title="Média de NPS")
fig.show()


Produto com a melhor média de NPS:
Product Category    Home
NPS                 5.01
Name: 3, dtype: object

Produto com a pior média de NPS:
Product Category    Electronics
NPS                        4.97
Name: 2, dtype: object


In [62]:
# passo 10: responder à pergunta "Qual categoria de produto mais e menos vendida?"

# agrupar os dados por categoria e calcular a quantidade total vendida
df_qtd_vendida = df.groupby("Product Category")["Quantity"].sum().reset_index()
df_qtd_vendida = df_qtd_vendida.sort_values(by="Quantity", ascending=False)
df_qtd_vendida


# encontrar a categoria mais vendida
cat_mais_vendida = df_qtd_vendida.iloc[0]

# encontrar a categoria menos vendida
cat_menos_vendida = df_qtd_vendida.iloc[-1]

# exibir resultados
print("\nCategoria mais vendida:")
print(cat_mais_vendida)

print("\nCategoria menos vendida:")
print(cat_menos_vendida)

# gráfico de quantidade vendida por categoria
fig = px.bar(df_qtd_vendida, x="Product Category", y="Quantity", title="Quantidade vendida por categoria",
             labels={"Product Category": "Categoria", "Quantity": "Quantidade Vendida por Categoria"},
             color="Quantity", color_continuous_scale=px.colors.sequential.Viridis)

fig.update_layout(xaxis_title="Product Category", yaxis_title="Quantidade Vendida")
fig.show()


Categoria mais vendida:
Product Category    Clothing
Quantity              225322
Name: 1, dtype: object

Categoria menos vendida:
Product Category      Home
Quantity            149698
Name: 3, dtype: object


In [63]:
# passo 11: responder à pergunta "Qual categoria mais e menos cara?"

# calculo da média dos preços para cada categoria
media_por_cat = df.groupby("Product Category")["Product Price"].mean().round(2).reset_index()

# identificar a categoria mais cara e a menos cara
cat_mais_cara = media_por_cat.loc[media_por_cat["Product Price"].idxmax()]
cat_menos_cara = media_por_cat.loc[media_por_cat["Product Price"].idxmin()]

# resultados
print("categoria mais cara:")
print(cat_mais_cara)

print("categoria menos cara:")
print(cat_menos_cara)

# criando grafico
fig = px.bar(media_por_cat, x="Product Category", y="Product Price", title="Média dos preços por categoria",
             labels={"Product Category": "Categoria", "Product Price": "Média de Preços"})

fig.show()

categoria mais cara:
Product Category      Home
Product Price       254.84
Name: 3, dtype: object
categoria menos cara:
Product Category    Clothing
Product Price         254.45
Name: 1, dtype: object


In [64]:
# passo 12: responder à pergunta "Qual o canal ideal para vender determinado tipo de produto (considerando genero e idade)?"

# agrupar dados por Categoria, Genero, Idade e Canal de Venda, e calcular a média de NPS
df_grouped = df.groupby(["Product Category", "Gender", "Customer Age ", "Source"])["NPS"].mean().round(2).reset_index()

# qual o melhor canal de venda para cada categoria, considerando genero e idade
melhor_canal = df_grouped.loc[df_grouped.groupby("Product Category")["NPS"].idxmax()]
melhor_canal

# filtrar os dados para o genero "Male"
melhor_canal_homem = melhor_canal[melhor_canal["Gender"] == "Male"]

# gráfico para o genero "Male"
fig_homem = px.bar(melhor_canal_homem, x="Product Category", y="NPS", color="Source", title="Melhor Canal de venda por categoria (genero: homem)",
                  labels={"Product Category": "Categoria do Produto", "NPS": "NPS", "Source": "Canal de Venda"})
fig_homem.update_layout(barmode="stack")

# filtrar os dados para o genero "Female"
melhor_canal_mulher = melhor_canal[melhor_canal["Gender"] == "Female"]

# gráfico para o genero "Male"
fig_mulher = px.bar(melhor_canal_mulher, x="Product Category", y="NPS", color="Source", title="Melhor Canal de venda por categoria (genero: mulher)",
                  labels={"Product Category": "Categoria", "NPS": "NPS", "Source": "Canal de Venda"})
fig_mulher.update_layout(barmode="stack")

# exibir graficos
fig_homem.show()
fig_mulher.show()

# informações retiradas da análise
- Quais produtos mais vendidos considerando os últimos 3 anos?
Os produtos mais vendidos entre 2021 e 2023 são:
Clothing: 164688 unidades vendidas
Books: 163000 unidades vendidas
Eletronics: 109328 unidades vendidas
Home: 108509 unidades vendidas

- Qual o produto mais caro e o mais barato?
O produto mais caro custa: 500
O produto mais barato custa: 10

- Qual a categoria de produto mais vendida e menos vendida?
clothing: 225322 unidades vendidas
Books: 223876 unidades vendidas
Eletronics: 150828 unidades vendidas
Home: 149698 unidades vendidas

categoria mais vendida: Clothing
categoria menos vendida: Home

- Qual a categoria mais cara e a mais barata? 
Média dos preços por categoria:
Home: 254,84
Eletronics: 254,72
Books: 254,71
Clothing: 254,45

categoria mais cara: Home
categoria mais barata: Clothing

- Qual o protudo com o melhor e o pior NPS?
Nesta análise, primeiramente foi identificada a média de NPS para cada categoria.

Resultado:
Home: 5.01 
Books: 5
Clothing: 4.98
Eletronics: 4.97

melhor NPS: Home
pior NPS: Eletronics

- Analisando a base de dados, qual o tipo de público (considerando gênero e idade) e o canal ideal para vender determinado tipo de produto?

- melhor canal de vendas:
	- Clothing: Organic Search
	- Books: Organic Search
	- Eletronics: Facebook Campaign
	- Home: Facebook Campaign
 
- melhor categoria por gênero:
	- Clothing: Female
	- Books: Male
	- Eletronics: Male
	- Home: Male